### Collaborative Filtering
```
user-based
item-based
```

#### Step1: 特徵處理

In [2]:
import numpy as np
import pandas as pd

In [3]:
ratings_df = pd.read_csv('./ratings_small.csv')
ratings_df.head(),ratings_df.shape

(   userId  movieId  rating   timestamp
 0       1       31     2.5  1260759144
 1       1     1029     3.0  1260759179
 2       1     1061     3.0  1260759182
 3       1     1129     2.0  1260759185
 4       1     1172     4.0  1260759205,
 (100004, 4))

In [4]:
ratings_df.nunique()

userId         671
movieId       9066
rating          10
timestamp    78141
dtype: int64

In [5]:
#依時間排序
ratings_df = ratings_df.sort_values('timestamp')
ratings_df.head()

,userId,movieId,rating,timestamp
52635,383,21,3.0,789652009
52641,383,47,5.0,789652009
52684,383,1079,3.0,789652009
56907,409,21,5.0,828212412
56909,409,25,4.0,828212412


In [6]:
#將userId,movieId分別對應到 0-N(num of users) /M(num of movies) 的整數
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

user_ids = user_encoder.fit_transform(ratings_df.userId)
movie_ids = movie_encoder.fit_transform(ratings_df.movieId)

In [7]:
#將資料分為訓練集和測試集(0.8/0.2)
num_train = int(len(user_ids)*0.8)

train_user_ids = user_ids[:num_train]
train_moviw_ids = movie_ids[:num_train]
train_ratings = ratings_df.rating.values[:num_train]
val_user_ids = user_ids[num_train:]
val_movie_ids = movie_ids[num_train:]
val_ratings = ratings_df.rating.values[num_train:]

train_user_ids.shape,train_moviw_ids.shape,train_ratings.shape,val_user_ids.shape,val_movie_ids.shape,val_ratings.shape

((80003,), (80003,), (80003,), (20001,), (20001,), (20001,))

In [8]:
#初始化user-movie matrix
num_users = user_ids.max()+1
num_movies = movie_ids.max()+1
user2movie = np.zeros([num_users, num_movies])
user2movie[train_user_ids,train_moviw_ids] = train_ratings
user2movie

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

#### Step2: 計算公式

$$
r = {\sum ^n _{i=1}(X_i- \bar X)(Y_i- \bar Y) \over \sqrt{\sum^n_{i=1}(X_i- \bar X)^2} \sqrt{\sum^n_{i=1}(Y_i- \bar Y)^2}}
$$
https://zh.wikipedia.org/wiki/%E7%9A%AE%E5%B0%94%E9%80%8A%E7%A7%AF%E7%9F%A9%E7%9B%B8%E5%85%B3%E7%B3%BB%E6%95%B0

In [9]:
# 皮爾森相似度
def pearson_correlation(x,y): #(x,y兩個使用者對不同電影的偏好向量)
    
    #只考慮兩個使用者共同評分過的電影
    filt = (x!=0) * (y!=0) #只考慮兩個都是True的狀況

    #計算平均，不考率為0的數
    x_mean = x.sum() / x[x!=0].shape
    y_mean = y.sum() / y[y!=0].shape

    x = x[filt]
    y = y[filt]

    corr = np.sum((x - x_mean) * (y-y_mean)) / (np.sum((x-x_mean) ** 2) ) ** 0.5
    return corr

In [11]:
#計算相似度
def compute_similarity_matrix(user2movie):
    #記錄每個使用者的相似度
    similarity_matrix = np.zeros([num_users,num_users])

    for i in range(len(user2movie)):
        for j in range(i, len(user2movie)):
            #皮爾森
            corr = pearson_correlation(user2movie[i],user2movie[j])

            similarity_matrix[i,j] = corr
            similarity_matrix[j,i] = corr
    return similarity_matrix

/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/1798362575.py:14: RuntimeWarning: invalid value encountered in double_scalars
  corr = np.sum((x - x_mean) * (y-y_mean)) / (np.sum((x-x_mean) ** 2) ) ** 0.5
/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/1798362575.py:9: RuntimeWarning: invalid value encountered in true_divide
  y_mean = y.sum() / y[y!=0].shape
/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/1798362575.py:8: RuntimeWarning: invalid value encountered in true_divide
  x_mean = x.sum() / x[x!=0].shape


array([[ 3.86652299,         nan,         nan, ...,  0.64864865,
                nan, -0.9173913 ],
       [        nan,  7.80940728, -0.03779505, ..., -0.70626998,
        -1.90946134,  1.24000629],
       [        nan, -0.03779505,  5.24497892, ...,  1.67588477,
        -0.19681608,  0.83829478],
       ...,
       [ 0.64864865, -0.70626998,  1.67588477, ...,  5.51655984,
         1.19354839,  0.80949707],
       [        nan, -1.90946134, -0.19681608, ...,  1.19354839,
         6.69617127,  0.55049151],
       [-0.9173913 ,  1.24000629,  0.83829478, ...,  0.80949707,
         0.55049151,  8.15262028]])

In [12]:
similarity_matrix = compute_similarity_matrix(user2movie)
similarity_matrix[:5]

/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/1798362575.py:14: RuntimeWarning: invalid value encountered in double_scalars
  corr = np.sum((x - x_mean) * (y-y_mean)) / (np.sum((x-x_mean) ** 2) ) ** 0.5
/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/1798362575.py:9: RuntimeWarning: invalid value encountered in true_divide
  y_mean = y.sum() / y[y!=0].shape
/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/1798362575.py:8: RuntimeWarning: invalid value encountered in true_divide
  x_mean = x.sum() / x[x!=0].shape


array([[ 3.86652299,         nan,         nan, ...,  0.64864865,
                nan, -0.9173913 ],
       [        nan,  7.80940728, -0.03779505, ..., -0.70626998,
        -1.90946134,  1.24000629],
       [        nan, -0.03779505,  5.24497892, ...,  1.67588477,
        -0.19681608,  0.83829478],
       [ 0.0718748 , -0.18334823,  0.54025835, ...,  1.0069077 ,
         0.47249985,  1.00727467],
       [-0.09      ,  0.22200195, -0.26790482, ...,  0.91732772,
        -1.52170388, -0.66359591]])

In [13]:
#將跟自己的相似度（對角線）、nan值 設為0
similarity_matrix[np.arange(num_users), np.arange(num_users)] = 0
similarity_matrix[np.isnan(similarity_matrix)] = 0
similarity_matrix[:5]

array([[ 0.        ,  0.        ,  0.        , ...,  0.64864865,
         0.        , -0.9173913 ],
       [ 0.        ,  0.        , -0.03779505, ..., -0.70626998,
        -1.90946134,  1.24000629],
       [ 0.        , -0.03779505,  0.        , ...,  1.67588477,
        -0.19681608,  0.83829478],
       [ 0.0718748 , -0.18334823,  0.54025835, ...,  1.0069077 ,
         0.47249985,  1.00727467],
       [-0.09      ,  0.22200195, -0.26790482, ...,  0.91732772,
        -1.52170388, -0.66359591]])

In [16]:
#user-base協同過濾
def compute_ucf(user2movie, similarity_matrix):
    '''
    compute prediction scores for all movies
    '''
    #yk
    mean_ratings = np.sum(user2movie,axis=1)/(user2movie != 0).sum(axis=1)
    
    #ykj - yk 每個電影減掉該使用者的平均評分（計算喜好程度)
    #user2movie=>(num_users, num_movies), mean_ratings=>(num_users,)-->(num_users,1)維度轉換
    user2movie_diff = user2movie - np.expand_dims(mean_ratings,axis=1)

    #每個使用者的相似度相加
    sim_sum = np.sum(np.abs(similarity_matrix),axis=1)
    #把原本是0的值 設為零
    user2movie_diff[np.where(user2movie == 0)] = 0
    
    #compute weighted sum of rating diff (num_users, num_movies)
    weighted_sum = np.matmul(similarity_matrix,user2movie_diff) / np.expand_dims(sim_sum,1)
    #得到預測分數
    scores = weighted_sum + np.expand_dims(mean_ratings,1)
    return scores
predictions = compute_ucf(user2movie, similarity_matrix)
predictions[:10]

/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/2518388212.py:7: RuntimeWarning: invalid value encountered in true_divide
  mean_ratings = np.sum(user2movie,axis=1)/(user2movie != 0).sum(axis=1)
/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/2518388212.py:19: RuntimeWarning: invalid value encountered in true_divide
  weighted_sum = np.matmul(similarity_matrix,user2movie_diff) / np.expand_dims(sim_sum,1)


array([[2.60070326, 2.49485163, 2.57093654, ..., 2.55      , 2.55      ,
        2.55      ],
       [3.53444596, 3.45781463, 3.46025568, ..., 3.48684211, 3.48684211,
        3.48684211],
       [3.60123031, 3.5589303 , 3.53456402, ..., 3.56862745, 3.56862745,
        3.56862745],
       ...,
       [3.98011337, 3.85742607, 3.80717723, ..., 3.86637931, 3.86637931,
        3.86637931],
       [3.93009418, 3.70764736, 3.69716432, ..., 3.75555556, 3.75555556,
        3.75555556],
       [3.83231515, 3.68569043, 3.63715838, ..., 3.69565217, 3.69565217,
        3.69565217]])

In [17]:
predictions.shape

(671, 9066)

#### 計算ＮＤＣＧ
- NDCG 能夠較為公平的比較不同長度推薦序列的好壞

DCG (Dsicounted Cumulative Gain)**
    - 衡量推薦出的物品順序符合使用者喜好的程度
    - 考慮物品相關分數（使用者評分）以及推薦結果的順序
$$DCG_p=\sum^p_{i=1}{2相關分數-1 \over \log_2(i+1)}$$
    
NDCG
    - 衡量目前推薦順序和最佳的推薦順序的差異
$$NDGC_p={DCG_p \over 最佳DCG_p}$$

In [15]:
def dcg_at_k(r, k):
    '''
    Compute DCG
    args:
        r: np.array, to be evaluated
        k: int, number of entries to be considered
    
    returns:
        dcg: float, computed dcg
        
    '''
    r = r[:k]
    dcg = np.sum(r / np.log2(np.arange(2, len(r) + 2)))
    return dcg

def ndcg_at_k(r, k, method=0):
    '''
    Compute NDCG
    args:
        r: np.array, to be evaluated
        k: int, number of entries to be considered
    
    returns:
        dcg: float, computed ndcg
        
    '''
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)

    return dcg_at_k(r, k) / dcg_max
    
# compute average ndcg for all users
def evaluate_prediction(predictions):
    '''
    Return the average ndcg for each users
    args:
        predictions: np.array user-item predictions
    returns:
        ndcg: float, computed NDCG
    '''
    ndcgs = []
    # iterate
    for target_user in np.unique(val_user_ids):
        # get movie ids and ratings associated with the target user.
        target_val_movie_ids = val_movie_ids[val_user_ids == target_user] 
        target_val_ratings = val_ratings[val_user_ids == target_user] 
        
        # compute ndcg for this user
        ndcg = ndcg_at_k(target_val_ratings[np.argsort(-predictions[target_user][target_val_movie_ids])], k=30)
        ndcgs.append(ndcg)
    ndcg = np.mean(ndcgs)
    return ndcg

In [18]:
evaluate_prediction(predictions)

0.8332450543581469

#### item-based 基於物品協同過濾

In [20]:
def compute_item_similarity_matrix(user2movie):
    #用矩陣運算計算相似度
    x_mean = user2movie.sum(axis=0) / (user2movie!=0).sum(axis=0)
    filt = (user2movie == 0)

    ratings_diff = user2movie-np.expand_dims(x_mean, axis = 0)
    ratings_diff[filt] = 0

    similarity_matrix = np.matmul(ratings_diff.T, ratings_diff) / (np.matmul(ratings_diff.T ** 2, (ratings_diff != 0 )) * np.matmul(ratings_diff.T ** 2, (ratings_diff != 0)).T) ** 0.5

    return similarity_matrix
similarity_matrix = compute_item_similarity_matrix(user2movie)
similarity_matrix[:10]

/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/1061981605.py:3: RuntimeWarning: invalid value encountered in true_divide
  x_mean = user2movie.sum(axis=0) / (user2movie!=0).sum(axis=0)
/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/1061981605.py:9: RuntimeWarning: invalid value encountered in true_divide
  similarity_matrix = np.matmul(ratings_diff.T, ratings_diff) / (np.matmul(ratings_diff.T ** 2, (ratings_diff != 0 )) * np.matmul(ratings_diff.T ** 2, (ratings_diff != 0)).T) ** 0.5


array([[ 1.        ,  0.32949699,  0.14883085, ...,         nan,
                nan,         nan],
       [ 0.32949699,  1.        ,  0.11771157, ...,         nan,
                nan,         nan],
       [ 0.14883085,  0.11771157,  1.        , ...,         nan,
                nan,         nan],
       ...,
       [ 0.75689798,  0.79614979,  0.71287669, ...,         nan,
                nan,         nan],
       [-0.21912884,  0.5874723 ,  0.34578862, ...,         nan,
                nan,         nan],
       [ 0.02433877,  0.43192313, -0.08695837, ...,         nan,
                nan,         nan]])

In [21]:
similarity_matrix[np.arange(num_movies), np.arange(num_movies)] = 0
similarity_matrix[np.isnan(similarity_matrix)] = 0
similarity_matrix

array([[0.        , 0.32949699, 0.14883085, ..., 0.        , 0.        ,
        0.        ],
       [0.32949699, 0.        , 0.11771157, ..., 0.        , 0.        ,
        0.        ],
       [0.14883085, 0.11771157, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [22]:
#item-based
def compute_icf(user2movie, similarity_matrix):
    '''
    Compute prediction scores for all movies with item-based CF.
    args:
        user2movie: np.array, user-movie rating matrix
    returns:
        scores: np.array, predicted scores of each video for the target user
    '''

    # compute mean rating yk, ignoring zero entries, shape:(num_movies)
    mean_ratings = np.sum(user2movie, axis=0) / (user2movie != 0 ).sum(axis=0)
    
    # compute ykj - yk, shape:(num_users, num_movies)
    user2movie_diff = user2movie - np.expand_dims(mean_ratings, axis=0)
    
    # compute sum of similarities Σsimik, (num_movies,)
    sim_sum = np.sum(np.abs(similarity_matrix), axis=1)
    
    # don't sum the unknown entries, set them to 0
    user2movie_diff[np.where(user2movie == 0) ] = 0
    
    # compute weighted sum of rating diff (num_users, num_movies)
    weighted_sum = np.matmul(user2movie_diff, similarity_matrix) / np.expand_dims(sim_sum, axis=0)

    # add weighted sum to mean ratings
    scores =  weighted_sum + np.expand_dims(mean_ratings, 0)
    
    return  scores

predictions = compute_icf(user2movie, similarity_matrix) 
predictions[:10]

/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/4108485018.py:11: RuntimeWarning: invalid value encountered in true_divide
  mean_ratings = np.sum(user2movie, axis=0) / (user2movie != 0 ).sum(axis=0)
/var/folders/ng/5b7r4nd52wl4987p44qbwlfw0000gn/T/ipykernel_52199/4108485018.py:23: RuntimeWarning: invalid value encountered in true_divide
  weighted_sum = np.matmul(user2movie_diff, similarity_matrix) / np.expand_dims(sim_sum, axis=0)


array([[3.91995938, 3.36217492, 3.19289318, ...,        nan,        nan,
               nan],
       [3.92128976, 3.3640285 , 3.19388683, ...,        nan,        nan,
               nan],
       [3.91963665, 3.36367295, 3.19216674, ...,        nan,        nan,
               nan],
       ...,
       [3.92075554, 3.36605949, 3.1937442 , ...,        nan,        nan,
               nan],
       [3.92067407, 3.36313627, 3.1926433 , ...,        nan,        nan,
               nan],
       [3.92048034, 3.36408442, 3.19267732, ...,        nan,        nan,
               nan]])

In [23]:
evaluate_prediction(predictions)

0.8721779505007486